In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

model_name = "../../../../projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4"
model_name = os.path.abspath(model_name)
print(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only = True, trust_remote_code = False)
model = AutoModelForCausalLM.from_pretrained(model_name, local_files_only = True, trust_remote_code = False, torch_dtype=torch.bfloat16)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("cuda available:", torch.cuda.is_available())
print("device:", device)
print("done")


/home/sheju347/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


cuda available: True
device: cuda
done


In [3]:

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)

In [9]:

example = {"question": "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complications and has not had any medical problems up until this time. However, for the past 4 days, he has been fussy, is regurgitating all of his feeds, and his vomit is yellow in color. On physical exam, the child's abdomen is minimally distended but no other abnormalities are appreciated. Which of the following embryologic errors could account for this presentation?",
        "choices": {
        "A": "Abnormal migration of ventral pancreatic bud",
        "B": "Complete failure of proximal duodenum to recanalize",
        "C": "Abnormal hypertrophy of the pylorus",
        "D": "Failure of lateral body folds to move ventrally and fuse in the midline"
        }
          }

# example = {"question": """Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38°C (100.4°F), pulse is 93/min, and blood pressure is 125/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:
# Hemoglobin count 14 g/dL
# Leukocyte count 16,400/mm3
# Segmented neutrophils 56%
# Eosinophils 11%
# Lymphocytes 31%
# Monocytes 2%
# Platelet count 260,000/mm3
# Erythrocyte sedimentation rate 68 mm/h
# Serum
# Urea nitrogen 25 mg/dL
# Creatinine 4.2 mg/dL
# Renal biopsy shows intravascular spindle-shaped vacuoles. Which of the following is the most likely cause of this patient's symptoms?""",
#            "choices": {
#             "A": "Renal papillary necrosis",
#             "B": "Cholesterol embolization",
#             "C": "Eosinophilic granulomatosis with polyangiitis",
#             "D": "Polyarteritis nodosa"
#             }
#         }


# example = {"question": "A microbiologist is studying the emergence of a virulent strain of the virus. After a detailed study of the virus and its life cycle, he proposes a theory: Initially, a host cell is co-infected with 2 viruses from the same virus family. Within the host cell, concomitant production of various genome segments from both viruses occurs. Ultimately, the different genome segments from the viruses are packaged into a unique and novel virus particle. The newly formed virus particle is both stable and viable and is a new strain from the virus family that caused the outbreak of infection. Which of the following viruses is capable of undergoing the above-mentioned process?",        "choices": {
# "A": "Epstein-Barr virus",
# "B": "Human immunodeficiency virus",
# "C": "Rotavirus",
# "D": "Vaccinia virus"
#         }
# }

prompt_eval_context_bare = f'''
{{context}} \n
{{question}} \n
{{choices}}
'''
            
formated_choices = format_choices(example["choices"])

# question = "cause of type 2 diabetes"
# context = "The cause of type 2 diabetes is lack of activity and excessive intake of sugar"
question = example["question"]

contexts = [
            # this is a CoT from UltraMedical, analyze option A, can help model get right answer
            "Abnormal migration of ventral pancreatic bud - This can lead to conditions such as annular pancreas, where the pancreas forms a ring around the duodenum and can cause duodenal obstruction. This would present with symptoms similar to what is described, including vomiting, which can be yellow if it contains bile.",
            
            # this is a CoT from UltraMedical, analyze option B, can't help model get right answer, still choose B
            #"Complete failure of proximal duodenum to recanalize - This would cause duodenal atresia, which presents with bilious vomiting within the first few days of life. While the vomit described is yellow (suggestive of bile), the timing and presentation don't align perfectly with duodenal atresia, as the symptoms would have likely appeared earlier than 3 weeks of age.",

            # "Abnormal migration of the ventral pancreatic bud can lead to bilious vomiting",
    
            # "Think carefully of each answer, consider the infant is 3-week-old",
            # "Complete failure of proximal duodenum to recanalize is typically presented immediately after birth, not 3 weeks",
           #"Aberrant insertion of the common bile duct into an accessory pancreatic duct: cholangiographic demonstration. Variations in the pancreatic ductal anatomy result from abnormalities in embryological development of the pancreas from ventral and dorsal buds. We describe a case in which cholangiography demonstrated insertion of the common bile duct into an aberrant pancreatic duct. To our knowledge, this finding has not been previously reported in the literature.",
           #"Clinical relevance of an unfused pancreatic duct system. In man, the main pancreatic duct is normally derived from ventral and dorsal embryological buds of the pancreas. In a minority of people, failure of fusion of the two buds results in separate drainage of the dorsal and ventral pancreas, so that the accessory duct provides the main drainage for the gland. Patients with this anomaly demonstrated at endoscopic retrograde pancreatography (ERP) have been investigated to assess whether non-fusion of the main pancreatic duct predisposes to the development of pancreatitis. A failure of fusion of the pancreatic ducts was seen in 21 out of 449 (4.7%) successful pancreatograms; four of these 21 patients had definite clinical evidence of pancreatitis and two patients had possible pancreatic disease, but in the remainder the anomaly was not considered to be clinically relevant. An abnormal pancreatogram suggesting pancreatitis was present in 116 out of the 428 patients (27.1%) with a normally fused duct system. The anomaly was found as frequently in the whole series as it was seen in patients with pancreatitis. These findings suggest that embryological failure of pancreatic duct fusion does not predispose to the development of pancreatitis. However, the presence of this anomaly may lead to misinterpretation of ultrasonographic and CT scan findings.",
           #"""migration of ventral pancreatic bud - This leads to annular pancreas, where pancreatic tissue encircles the duodenum and can cause duodenal obstruction with bilious vomiting."""           ,
           
# """Abnormal hypertrophy of the pylorus refers to the thickening of the pyloric muscle, specifically in the pylorus, which is the narrow passage between the stomach and the small intestine. This thickening, also known as hypertrophic pyloric stenosis (HPS), can lead to a narrowing and obstruction of the pylorus, preventing food from entering the small intestine. 
# Key points about abnormal hypertrophy of the pylorus:
# Cause:
# The thickening of the pyloric muscle is the primary issue, causing a narrowing and obstruction. 
# Location:
# The pylorus, the passage between the stomach and small intestine, is affected. 
# Consequences:
# The obstruction can lead to vomiting, dehydration, and poor nutrition in infants. 
# Treatment:
# Treatment usually involves a surgical procedure called pyloromyotomy, where the thickened muscle is split to relieve the obstruction. """,

           
# """Abnormal migration of the ventral pancreatic bud, specifically in the context of annular pancreas, refers to a developmental anomaly where the ventral pancreatic bud, which normally migrates around the duodenum, fails to do so properly. This failure leads to a ring of pancreatic tissue encircling the duodenum, causing various complications. 
# Here's a more detailed explanation:
# Normal Development:
# During fetal development, the pancreas develops from two buds: the dorsal and ventral pancreatic buds. The ventral bud, which is also connected to the bile duct, normally migrates around the duodenum and fuses with the dorsal bud. 
# Abnormal Migration in Annular Pancreas:
# In annular pancreas, the ventral bud fails to migrate correctly. This can happen due to a few reasons: 
# Lack of rotation: The two lobes of the bilobed ventral bud may not rotate correctly around the duodenum. 
# Failure of the right bud to migrate: The right ventral bud might not migrate as expected. 
# Genetic factors: Genetic mutations in genes involved in cell migration, like IQGAP1 and NRCAM, could potentially contribute to abnormal migration. 
# Consequences of Abnormal Migration:
# The abnormal migration leads to the formation of a ring of pancreatic tissue around the duodenum, which can:
# Cause duodenal obstruction:
# The ring of tissue can constrict the duodenum, leading to blockage, particularly in infants. 
# Lead to other associated anomalies:
# Annular pancreas is often associated with other congenital anomalies like duodenal atresia, tracheoesophageal fistula, and trisomy 21. 
# Present with symptoms in adults:
# While many individuals with annular pancreas are asymptomatic, adults may experience symptoms like abdominal pain, peptic ulcer disease, and pancreatitis. 
# Diagnosis and Treatment:
# Annular pancreas is usually diagnosed during infancy or in childhood due to obstructive symptoms. Imaging techniques like MRCP and CT scans can help visualize the ring of pancreatic tissue. Treatment may involve surgical resection of the obstructing pancreatic tissue. 
# """,
           
           ]
# context = "The infant's presentation with fussiness, regurgitation of feeds, and yellow-colored vomit is suggestive of a gastrointestinal issue that is likely related to an embryologic development error. Let's analyze each of the provided options:\n\nA. Abnormal migration of ventral pancreatic bud - This can lead to conditions such as annular pancreas, where the pancreas forms a ring around the duodenum and can cause duodenal obstruction. "#"don't Tell the attending that he cannot fail to disclose this mistake"
context = ""
for c in contexts:
    context += c + "\n"

choices = example["choices"]
prompt = prompt_eval_context_bare.format(question = question, context = context, choices = formated_choices)

messages = [{"role": "user", "content": f"{prompt}"}]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens = 1024,
        do_sample=True,
        temperature = 0.0000000000001,
        
        # Need these two?
        # attention_mask=inputs["attention_mask"],  # Use attention mask
        # pad_token_id=tokenizer.eos_token_id,  # Set pad token ID
        # do_sample = False
    )
text = tokenizer.batch_decode(outputs)[0]
# print("outputs text:", text)
text = text.split("<|assistant|>")[-1]
print("final response:\n", text)


final response:
  The symptoms described for the infant, such as fussiness, regurgitation of feeds, and yellow vomit (which may indicate the presence of bile), suggest a possible obstruction in the gastrointestinal tract. Among the options provided, the one that is most likely to cause such symptoms is an abnormality related to the pancreas.

[A]. Abnormal migration of the ventral pancreatic bud can lead to the formation of an annular pancreas, which is a ring of pancreatic tissue that can encircle and potentially obstruct the duodenum. This would cause symptoms similar to those described, including vomiting.

[B]. Complete failure of the proximal duodenum to recanalize would lead to duodenal atresia, which is a congenital obstruction of the duodenum. This condition typically presents with bilious vomiting shortly after birth, not at 3 weeks of age.

[C]. Abnormal hypertrophy of the pylorus, known as pyloric stenosis, typically presents later in infancy, around 2-8 weeks of age, with p